In [ ]:
import pandas as pd
from langchain_openai_augmentation import ContextGenerator
import time
import os

# Generator 초기화
generator = ContextGenerator(model='gpt-4.1-mini')

# 체크포인트 파일 경로
checkpoint_path = r'C:\Users\user\Desktop\SKN_AFTER_STUDY\data\augmentation_checkpoint.xlsx'

# 기존 체크포인트 로드 함수
def load_checkpoint():
    if os.path.exists(checkpoint_path):
        existing_df = pd.read_excel(checkpoint_path)
        print(f"체크포인트 파일 로드: {len(existing_df)}개의 기존 데이터")
        return existing_df
    else:
        print("새로운 증강 시작")
        return pd.DataFrame()

# 체크포인트 저장 함수
def save_checkpoint(augmented_data):
    checkpoint_df = pd.DataFrame(augmented_data)
    checkpoint_df.to_excel(checkpoint_path, index=False)
    print(f"체크포인트 저장: {len(augmented_data)}개")

# 각 중분류-소분류별 24개씩 증강하는 함수 (순환 증강 로직)
def augment_data_by_category_resume(df, target_count=24):
    # 기존 증강 데이터 로드
    existing_augmented_df = load_checkpoint()
    augmented_data = existing_augmented_df.to_dict('records') if len(existing_augmented_df) > 0 else []
    
    # 이미 완성된 카테고리 찾기
    completed_categories = set()
    category_progress = {}
    
    if len(existing_augmented_df) > 0 and 'category1' in existing_augmented_df.columns:
        existing_stats = existing_augmented_df.groupby(['category1', 'category2']).size().reset_index(name='count')
        for _, row in existing_stats.iterrows():
            key = (row['category1'], row['category2'])
            category_progress[key] = row['count']
            if row['count'] >= target_count:
                completed_categories.add(key)
                print(f"이미 완료: {row['category1']} - {row['category2']} ({row['count']}개)")
    
    # 중분류-소분류별로 그룹화
    grouped = df.groupby(['category1', 'category2'])
    
    for (category1, category2), group in grouped:
        category_key = (category1, category2)
        
        # 이미 완료된 카테고리 건너뛰기
        if category_key in completed_categories:
            continue
            
        print(f"\n처리 중: {category1} - {category2} ({len(group)}개)")
        
        current_progress = category_progress.get(category_key, 0)
        print(f"  기존 진행: {current_progress}개")
        
        if current_progress >= target_count:
            print(f"  이미 완료됨")
            continue
            
        current_count = len(group)
        
        # 증강된 컨텍스트들을 저장할 리스트 (순환 참조용)
        augmented_contexts = []
        
        # 기존에 생성된 증강 데이터가 있다면 로드
        if current_progress > 0 and len(existing_augmented_df) > 0:
            existing_data = existing_augmented_df[
                (existing_augmented_df['category1'] == category1) & 
                (existing_augmented_df['category2'] == category2)
            ].copy()
            
            # 기존 데이터를 순서대로 정렬
            existing_data = existing_data.sort_values(['augmentation_index'], na_position='first')
            
            for _, row in existing_data.iterrows():
                augmented_contexts.append(row['generator_context'])
        
        # 원본 데이터 처리 (아직 처리하지 않은 것들만)
        if current_count >= target_count:
            selected_rows = group.iloc[:target_count]
        else:
            selected_rows = group
            
        start_idx = len(augmented_contexts)
        
        # 선택된 행들로 증강 수행 (아직 처리하지 않은 것들만)
        for i, (idx, row) in enumerate(selected_rows.iterrows()):
            if i < start_idx:  # 이미 처리된 경우 건너뛰기
                continue
                
            try:
                augmented_context = generator.create_context(
                    row['context'], 
                    category1, 
                    category2
                )
                
                augmented_data.append({
                    'generator_context': augmented_context,
                    'category1': category1,
                    'category2': category2,
                    'input_context': row['context'],
                    'original_index': idx,
                    'augmentation_index': None
                })
                
                augmented_contexts.append(augmented_context)
                current_progress += 1
                
                print(f"  원본 데이터 {i+1}/{len(selected_rows)} 완료")
                time.sleep(0.5)
                
                # 중간 저장 (10개마다)
                if len(augmented_data) % 10 == 0:
                    save_checkpoint(augmented_data)
                
            except Exception as e:
                print(f"  오류 발생 (인덱스 {idx}): {e}")
                continue
        
        # target_count 미만인 경우 순환 증강
        if current_progress < target_count:
            remaining_count = target_count - current_progress
            print(f"  추가 증강 필요: {remaining_count}개 (순환 증강 방식)")
            
            # 기존 증강 인덱스 찾기 (체크포인트에서 최대값)
            current_augmentation_index = 0
            if len(existing_augmented_df) > 0 and 'augmentation_index' in existing_augmented_df.columns:
                existing_additional = existing_augmented_df[
                    (existing_augmented_df['category1'] == category1) & 
                    (existing_augmented_df['category2'] == category2) & 
                    (existing_augmented_df['augmentation_index'].notna())
                ]
                if len(existing_additional) > 0:
                    current_augmentation_index = int(existing_additional['augmentation_index'].max()) + 1
            
            for i in range(remaining_count):
                try:
                    # 순환 참조: augmented_contexts에서 인덱스로 순환하며 선택
                    source_index = i % len(augmented_contexts)
                    source_context = augmented_contexts[source_index]
                    
                    new_augmented_context = generator.create_context(
                        source_context,
                        category1,
                        category2
                    )
                    
                    augmented_data.append({
                        'generator_context': new_augmented_context,
                        'category1': category1,
                        'category2': category2,
                        'input_context': source_context,
                        'original_index': None,
                        'augmentation_index': current_augmentation_index
                    })
                    
                    # 새로 생성된 컨텍스트를 리스트에 추가 (다음 순환에서 사용)
                    augmented_contexts.append(new_augmented_context)
                    current_progress += 1
                    current_augmentation_index += 1
                    
                    print(f"  추가 증강 {i+1}/{remaining_count} 완료 (소스: 인덱스 {source_index})")
                    time.sleep(0.5)
                    
                    # 중간 저장
                    if len(augmented_data) % 10 == 0:
                        save_checkpoint(augmented_data)
                        
                except Exception as e:
                    print(f"  추가 증강 오류: {e}")
                    continue
    
    # 최종 결과 DataFrame 생성 및 저장
    augmented_df = pd.DataFrame(augmented_data)
    save_checkpoint(augmented_data)
    return augmented_df

In [ ]:
# 데이터 로드 및 실행
df = pd.read_excel(r'C:\Users\user\Desktop\SKN_AFTER_STUDY\data\증강할데이터33.xlsx')
print("데이터 증강을 시작합니다...")
augmented_df = augment_data_by_category_resume(df, target_count=24)